In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from openai import OpenAI

# Configure OpenAI client for DeepInfra
client = OpenAI(
    base_url="https://api.deepinfra.com/v1/openai",
    api_key="0jxRzr6VTMJsMSnR2NomXgP0PKDkEEw5",
)

# Initialize Qdrant client
qdrant_client = QdrantClient(":memory:")  # Use in-memory for testing, or specify host

# Create collection with BGE model dimensions (768)
qdrant_client.create_collection(
    collection_name="my_collection",
    vectors_config=models.VectorParams(
        size=768,  # BAAI/bge-base-en-v1.5 outputs 768-dimensional vectors
        distance=models.Distance.COSINE,  # BGE models use cosine similarity
    )
)

# Generate embeddings using DeepInfra
texts = ["The food was delicious and the waiter...", "Another example text"]
vectors = []

for text in texts:
    response = client.embeddings.create(
        model="BAAI/bge-base-en-v1.5",
        input=text,
        encoding_format="float"  # Optional but explicit
    )
    vectors.append(response.data[0].embedding)

# Upload to Qdrant
qdrant_client.upload_points(
    collection_name="my_collection",
    points=[
        models.PointStruct(
            id=idx,
            vector=vector,
            payload={"text": text}
        )
        for idx, (vector, text) in enumerate(zip(vectors, texts))
    ]
)

In [9]:
# Generate query embedding
query = "restaurant experience"
response = client.embeddings.create(
    model="BAAI/bge-base-en-v1.5",
    input=query,
    encoding_format="float"
)
query_vector = response.data[0].embedding

# Search in Qdrant
hits = qdrant_client.query_points(
    collection_name="my_collection",
    query=query_vector,
    limit=3
).points

In [12]:
hits[0].payload

{'text': 'The food was delicious and the waiter...'}